In [251]:
import pandas as pd 
import utils
import plotly.graph_objects as go 
import numpy as np 
import instrument

In [316]:
pair = 'EUR_USD'
granularity = 'H1'
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [317]:
his_candles = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [318]:
df = his_candles.copy()

In [319]:
df.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
3994,2024-09-05T17:00:00.000000000Z,3638,1.11010,1.11073,1.10996,1.11067,1.11002,1.11065,1.10987,1.11060,1.11017,1.11081,1.11004,1.11074
3995,2024-09-05T18:00:00.000000000Z,3494,1.11071,1.11134,1.11025,1.11043,1.11064,1.11126,1.11018,1.11035,1.11078,1.11142,1.11032,1.11051
3996,2024-09-05T19:00:00.000000000Z,3914,1.11042,1.11078,1.11028,1.11038,1.11035,1.11070,1.11021,1.11030,1.11050,1.11087,1.11035,1.11045
3997,2024-09-05T20:00:00.000000000Z,1499,1.11038,1.11114,1.11038,1.11106,1.11030,1.11106,1.11030,1.11099,1.11047,1.11121,1.11047,1.11114
3998,2024-09-05T21:00:00.000000000Z,513,1.11102,1.11119,1.11090,1.11096,1.11089,1.11109,1.11079,1.11084,1.11114,1.11129,1.11102,1.11109


In [320]:
non_cols = ['time', 'volume']
non_cols

['time', 'volume']

In [321]:
mod_cols = [x for x in df.columns if x not in non_cols]
mod_cols

['mid_o',
 'mid_h',
 'mid_l',
 'mid_c',
 'bid_o',
 'bid_h',
 'bid_l',
 'bid_c',
 'ask_o',
 'ask_h',
 'ask_l',
 'ask_c']

In [322]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [323]:
df['time'] = pd.to_datetime(df['time'])

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   time    3999 non-null   datetime64[ns, UTC]
 1   volume  3999 non-null   int64              
 2   mid_o   3999 non-null   float64            
 3   mid_h   3999 non-null   float64            
 4   mid_l   3999 non-null   float64            
 5   mid_c   3999 non-null   float64            
 6   bid_o   3999 non-null   float64            
 7   bid_h   3999 non-null   float64            
 8   bid_l   3999 non-null   float64            
 9   bid_c   3999 non-null   float64            
 10  ask_o   3999 non-null   float64            
 11  ask_h   3999 non-null   float64            
 12  ask_l   3999 non-null   float64            
 13  ask_c   3999 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(12), int64(1)
memory usage: 437.5 KB


Plotting the Candles 

Getting the 200 EMA Line 

In [325]:
df['MA_200'] = df['mid_c'].rolling(window=200).mean()
df['MA_200'] = df['MA_200'].fillna(method='bfill')


C:\Users\cnama\AppData\Local\Temp\ipykernel_23440\1329749941.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['MA_200'] = df['MA_200'].fillna(method='bfill')


Fractals Strategy 

In [326]:
df['frac_top_bool'] = np.where(
    df['mid_h'] == df['mid_h'].rolling(5, center=True).max(), True, False
)

In [327]:
df['frac_top'] = np.where(
    df['mid_h'] == df['mid_h'].rolling(5, center=True).max(), df['mid_h'], None
)

In [328]:
#filling the subsequent rows with the previous fractal high till the next fractal high 
df['frac_top'] = df['frac_top'].ffill()

C:\Users\cnama\AppData\Local\Temp\ipykernel_23440\2781819301.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['frac_top'] = df['frac_top'].ffill()


In [329]:
df['frac_bottom_bool'] = np.where(
    df['mid_l'] == df['mid_l'].rolling(5, center=True).min(), True, False
)

In [330]:
df['frac_bottom'] = np.where(
    df['mid_l'] == df['mid_l'].rolling(5, center=True).min(), df['mid_l'], None
)

In [331]:
df['frac_bottom'] = df['frac_bottom'].ffill()

C:\Users\cnama\AppData\Local\Temp\ipykernel_23440\2781400002.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['frac_bottom'] = df['frac_bottom'].ffill()


In [332]:
df.shape

(3999, 19)

In [333]:
df.isnull().sum()

time                0
volume              0
mid_o               0
mid_h               0
mid_l               0
mid_c               0
bid_o               0
bid_h               0
bid_l               0
bid_c               0
ask_o               0
ask_h               0
ask_l               0
ask_c               0
MA_200              0
frac_top_bool       0
frac_top            4
frac_bottom_bool    0
frac_bottom         9
dtype: int64

In [334]:
#dropping the NaN values 
df.dropna(inplace=True)

In [335]:
df.shape

(3990, 19)

Both buy and sell scenarios 

In [336]:
def is_trade(row):
    if (row.mid_c > row.frac_top) & (row.mid_c > row.MA_200):
        return 1 #buy 
    
    if (row.mid_c < row.frac_bottom) & (row.mid_c < row.MA_200):
        return -1 #sell
    
def sl(row):
    if row.IS_TRADE == 1:
        return row['mid_c']-row['mid_l']
        

    if row.IS_TRADE == -1:
        return row['mid_c']-row['mid_l']

def tp(row):
    if row.IS_TRADE == 1:
        return row['SL'] * 1.5

    if row.IS_TRADE == -1:
        return row['SL'] * 1.5
    
#checking the gains 

def gain(row):
    if row.IS_TRADE == 1:
        if row['mid_h'] >= (row['mid_c'] + row['TP']):  # Take-profit hit
            return row['TP']
        elif row['mid_l'] <= (row['mid_c'] - row['SL']):  # Stop-loss hit
            return -row['SL']
    
    elif row.IS_TRADE == -1:
        if row['mid_l'] <= (row['mid_c'] - row['TP']):  # Take-profit hit
            return row['TP']
        elif row['mid_h'] >= (row['mid_c'] + row['SL']):  # Stop-loss hit
            return -row['SL']
    return 0  # No gain/loss if no TP or SL hit

In [337]:
df['IS_TRADE'] = df.apply(is_trade, axis=1)
df['SL'] = df.apply(sl, axis = 1)
df['TP'] = df.apply(tp, axis=1)
df['GAIN'] = df.apply(gain, axis=1)


In [338]:
df.tail(40)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_c,MA_200,frac_top_bool,frac_top,frac_bottom_bool,frac_bottom,IS_TRADE,SL,TP,GAIN
3959,2024-09-04 06:00:00+00:00,5120,1.10579,1.10580,1.10474,1.10534,1.10572,1.10573,1.10466,1.10526,...,1.10541,1.111108,False,1.10564,False,1.10396,NaN,NaN,NaN,0.0
3960,2024-09-04 07:00:00+00:00,6672,1.10536,1.10629,1.10468,1.10560,1.10528,1.10621,1.10460,1.10553,...,1.10568,1.111078,True,1.10629,True,1.10468,NaN,NaN,NaN,0.0
3961,2024-09-04 08:00:00+00:00,6029,1.10559,1.10590,1.10496,1.10518,1.10552,1.10582,1.10489,1.10510,...,1.10526,1.111044,False,1.10629,False,1.10468,NaN,NaN,NaN,0.0
3962,2024-09-04 09:00:00+00:00,3833,1.10520,1.10587,1.10503,1.10533,1.10512,1.10579,1.10496,1.10526,...,1.10540,1.111007,False,1.10629,False,1.10468,NaN,NaN,NaN,0.0
3963,2024-09-04 10:00:00+00:00,3582,1.10534,1.10560,1.10500,1.10532,1.10526,1.10552,1.10492,1.10524,...,1.10539,1.110971,False,1.10629,False,1.10468,NaN,NaN,NaN,0.0
3964,2024-09-04 11:00:00+00:00,4379,1.10531,1.10540,1.10428,1.10504,1.10523,1.10532,1.10420,1.10496,...,1.10512,1.110933,False,1.10629,False,1.10468,NaN,NaN,NaN,0.0
3965,2024-09-04 12:00:00+00:00,5382,1.10504,1.10526,1.10400,1.10525,1.10496,1.10518,1.10392,1.10518,...,1.10532,1.110896,False,1.10629,True,1.10400,NaN,NaN,NaN,0.0
3966,2024-09-04 13:00:00+00:00,5846,1.10524,1.10593,1.10500,1.10576,1.10517,1.10586,1.10493,1.10564,...,1.10589,1.110862,False,1.10629,False,1.10400,NaN,NaN,NaN,0.0
3967,2024-09-04 14:00:00+00:00,12377,1.10576,1.10952,1.10574,1.10898,1.10564,1.10944,1.10564,1.10891,...,1.10905,1.110842,True,1.10952,False,1.10400,NaN,NaN,NaN,0.0
3968,2024-09-04 15:00:00+00:00,6170,1.10897,1.10918,1.10764,1.10809,1.10890,1.10911,1.10757,1.10802,...,1.10816,1.110823,False,1.10952,False,1.10400,NaN,NaN,NaN,0.0


In [339]:
trades = df.dropna()

In [340]:
trades.shape

(538, 23)

In [341]:
trades.GAIN.sum()*1000

np.float64(-321.48000000000263)